# MNIST en Deep Learning

Dans ce TP, nous allons construire des algorithmes de Deep Learning pour tenter de reconnaître des chiffres manuscrits.

## Chargement des données et transformation

Nous allons travailler sur la base de données MNIST qui contient 60000 images en niveaux de grille de résolution 28x28, représentant les 10 chiffres de 0 à 9, ainsi qu'un jeu de test de 10000 images. Tout d'abord, chargeons ce jeu de données.

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
import matplotlib
import matplotlib.pyplot as plt
import scipy
import scipy.ndimage
import numpy as np
from keras import backend as K
import keras

(X_train_base, y_train_base), (X_test_base, y_test_base) = mnist.load_data()


Visualisons quelques données.

In [ ]:
def plot_10_by_10_images(images):
    """ Plot 100 MNIST images in a 10 by 10 table. Note that we crop
    the images so that they appear reasonably close together.  The
    image is post-processed to give the appearance of being continued."""
    fig = plt.figure()
    images = [image[3:25, 3:25] for image in images]
    #image = np.concatenate(images, axis=1)
    for x in range(10):
        for y in range(10):
            ax = fig.add_subplot(10, 10, 10*y+x+1)
            ax.matshow(images[10*y+x+1], cmap = matplotlib.cm.binary)
            plt.xticks(np.array([]))
            plt.yticks(np.array([]))
    plt.show()

plot_10_by_10_images(X_train_base)

Maintenant que nous avons chargé les données, nous allons modifier la dimension de matrices, afin de les mettre sous une forme qui pourra être traitée par nos réseaux de neurones.

In [ ]:
subset=10000 #size
nb_classes=10
X_train = X_train_base[:subset].reshape(subset, 784)
X_test = X_test_base.reshape(10000, 784)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255
y_train = np_utils.to_categorical(y_train_base[:subset], nb_classes)
y_test = np_utils.to_categorical(y_test_base, nb_classes)


Nous avons maintenant des vecteurs à la place des images.

In [ ]:
print(X_train_base.shape)
print(X_train.shape)

## Construire un premier réseau de neurones

Construisons notre premier réseau de neurones.

Pour cela, nous allons créer un modèle Keras en utilisant l'api Sequential:
* __model = Sequential()__

Puis utiliser les méthodes suivantes de Keras pour ajouter des couches à ce modèle :

* __model.add(Dense(nbhiden, [input_shape=(nbinput,1)], [activation='activationfunction']))__ : ajoute une couche cachée au réseau de neurones. nbhidden est le nombre de neurones de la couche. [input_shape=(nbinput,)] correspond à la dimension de l'entrée, [activation='activationfunction'] est la fonction d'activation
* __model.add(Dropout(value))__ : applique un dropout à la couche, pour éviter le surapprentissage

In [ ]:
# A COMPLETER

# Créer le réseau suivant avec Keras :
# 784 dimensions en entrée => 12 dense => activation relu => 12 dense => activation relu => dropout 0.5 => 10 sorties (activation softmax)

model = None

# Compilation du modèle
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],  optimizer=sgd)

model.summary()

Ensuite, nous allons lancer l'apprentissage des paramètres.

In [ ]:
batch_size = 256
epochs=20
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,  verbose=1, validation_data=(X_test, y_test))

Nous vous laissons analyser les résultats. Ce réseau de neurones est-il performant ?

__A vous de jouer__ : essayez de créer un meilleur réseau de neurones, afin d'atteindre le meilleur résultat possible.

In [ ]:
# A COMPLETER

# Créer un meilleur réseau de neurones, et l'entraîner
# Objectif : avoir le meilleur résultat possible

Voyons ce que donne notre modèle sur un exemple.

In [ ]:
def plot_mnist_digit(image):
    """ Plot a single MNIST image."""
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(image, cmap = matplotlib.cm.binary)
    plt.xticks(np.array([]))
    plt.yticks(np.array([]))
    plt.show()
loss,acc = model.evaluate(X_test, y_test,  verbose=0)
index=800
print('The accuracy on the test set is ',(acc*100),'%')
plot_mnist_digit(X_test_base[index])
cl=model.predict_classes(X_test[index].reshape((1,784)))


print("le chiffre reconnu est: ", cl[0])
print("le chiffre à reconnaitre  est: ", np.argmax(y_test[index]))

## CNN : réseaux de neurones convolutionnels

Nous allons maintenant implémenter un réseau de neurones convolutionnel.

Pour cet exercice, vous allez avoir besoin des méthodes Keras suivantes, en plus de celles déjà vues précédemment :

* __model.add(Conv2D(filters, kernel_size=(size, size), [activation='activation_function'], [input_shape=input_shape]))__ : ajoute une couche de convolution au réseau de neurones. filters est le nombre de filtres de convolution. kernel_size est la taille des filtres de convolution.
* __model.add(MaxPooling2D(pool_size=(psize, psize)))__ : ajoute une couche de max pooling. pool_size est la taille de la fenêtre de pooling.

In [ ]:
img_rows, img_cols = 28, 28
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train[:subset].reshape(x_train[:subset].shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = keras.utils.to_categorical(y_train[:subset], nb_classes)
y_test = keras.utils.to_categorical(y_test, nb_classes)


In [ ]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
# A COMPLETER

# Créer le réseau suivant avec Keras :
# input_shape dimensions en entrée => 4 filtres de convolution 3x3 => activation relu =>
#    => maxpooling 2x2 => dropout 0.25 => flatten => 10 dense => dropout 0.5 =>
#    => activation relu => nb_classes sorties (activation softmax)

model = None

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],  optimizer=sgd)

model.summary()

In [ ]:
batch_size = 256
epochs=20
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,  verbose=1, validation_data=(x_test, y_test))

In [ ]:
def plot_mnist_digit(image):
    """ Plot a single MNIST image."""
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(image, cmap = matplotlib.cm.binary)
    plt.xticks(np.array([]))
    plt.yticks(np.array([]))
    plt.show()
loss,acc = model.evaluate(x_test, y_test,  verbose=0)
index=800
print('The accuracy on the test set is ',(acc*100),'%')
plot_mnist_digit(X_test_base[index])
cl=model.predict_classes(x_test[index].reshape((1,28,28,1)))


print("le chiffre reconnu est: ", cl[0])
print("le chiffre à reconnaitre  est: ", np.argmax(y_test[index]))


In [ ]:
# A COMPLETER

# Créer un meilleur réseau de neurones, et l'entraîner
# Objectif : avoir le meilleur résultat possible

## Bonus : Auto encodeur

L'auto-encodeur est un réseau de neurones qui compresse puis décompresse l'information. On l'entraîne en lui demandant de retrouver en sortie la même image que celle qu'il avait en entrée. Ici, l'information en entrée est en dimension 784 (28x28), et l'auto-encodeur va la compresser en dimension 2.

In [ ]:
encoding_dim = 2

model = Sequential()
model.add(Dense(encoding_dim, input_shape=(784,),activation='relu'))

model.add(Dense(784, activation='sigmoid'))
    
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',  optimizer=sgd)

model.summary()


__Question__ : dessiner l'architecture de ce réseau de neurones.

In [ ]:
(X_train, _), (X_test, _) = mnist.load_data()
subset=50000 #size
X_train = X_train[:subset].reshape(subset, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255

In [ ]:
model.fit(X_train, X_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))

In [ ]:
decoded_imgs = model.predict(X_test)

Affichons quelques images pour voir comment se comporte notre auto-encodeur.

In [ ]:
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

__A vous de jouer__ : essayez d'améliorer l'auto-encodeur.

In [ ]:
## A COMPLETER

# Réaliser un meilleur auto-encodeur